This Jupyter notebook provides some basic building blocks for analyzing Chronicling America Title Essay datasets. Please reach out with any comments, edits, suggestions, or questions.

--------

First we will import the necessary packages. To ease our analysis we rely heavily on two popular Python natural language processing frameworks: `nltk` and `spacy`. As part of this import step, we need to download the necessary tokenization and language models packages.

In [1]:
from bs4 import BeautifulSoup
import csv
import logging
import pandas as pd
import requests
import time
import nltk
import spacy
import numpy as np
from nltk.text import Text

# display max column width for descriptions
pd.set_option('display.max_colwidth', None)

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

!python3 -m spacy download en_core_web_sm

# load small language model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.4 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Next we'll read in the raw data. To make sure everything is working correctly, we'll print out the first few lines

In [3]:
df = pd.read_csv("chron_am_essays.csv")
df.head()

created_published  date  \
0               ['Abbeville, S.C. : Charles H. Allen, 1847-1869.']  1847   
1                                ['Abbeville, S.C. : Hugh Wilson']  1865   
2                          ['Abbeville, S.C. : Bonham and Perrin']  1884   
3         ['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 1869-1924.']  1869   
4  ['Abbeville, S.C. : W.A. Lee and Hugh Wilson, Jr., 1860-1869.']  1860   

  dates_of_publication  \
0            1847-1869   
1            1865-1865   
2            1884-1887   
3            1869-1924   
4            1860-1869   

                                                                                                                                                                                                                                                                                                                                                                                                                                                description  \
0                                                                                                              ['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']   
1                                                                                                         ['Weekly Began in July 1865. Ceased with Aug. 31, 1865. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Published during the suspension of the Abbeville press. Description based on: Vol. 1, no. 3 (Aug. 10, 1865). Latest issue consulted: Vol. 1, no. 6 (Aug. 31, 1865).']   
2                                                                                                                                  ['Weekly Began Oct. 1, 1884; ceased in 1887. Cf. Watchman and southron, July 20, 1887. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Description based on: Vol. 1, no. 1 (Oct. 1, 1884). Latest issue consulted: Vol. 3, no. 38 (June 28, 1887).']   
3  ['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began with Oct. 1, 1869; ceased with Feb. 13, 1924. Title varies slightly. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Formed by the union of: Abbeville press and: Abbeville banner. Description based on: Vol. 17, no. 23 (Oct. 1, 1869). Latest issue consulted: Feb. 13, 1924. Press and banner (DLC)  2013228796 (OCoLC)841312126']   
4                                                                                  ['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, no. 22 (Sept. 24, 1869). Suspended <1864>-Aug. 1865. Resumed with Sept. 7, 1865 issue. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville banner (DLC)sn 85026945 (OCoLC)12795764 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

We next create a small Python function that removes the html tags from the `essay` column:

In [4]:
def stripped_strings(text):
    try:
        return (' '.join(BeautifulSoup(text).stripped_strings))
    except TypeError as e:
        print(e)


df['essay'] = [stripped_strings(text) for text in df['essay'] ]
    

object of type 'float' has no len()


In [5]:
df.head()

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url
0,"['Abbeville, S.C. : Charles H. Allen, 1847-1869.']",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']","For nearly a century, the Abbeville Press and Banner (1844-1925) reported on international, national, and local news for the citizens of Abbeville County, located in the upstate region of South Carolina. The N.W. Ayer & Son’s American Newspaper Annual described Abbeville County simply as a “cotton growing district,” which while true on the surface is also slightly misleading. Although the cultivation of cotton had indeed dominated the county’s economy for most of the 19 th century, in the 1890s Abbeville was undergoing a period of economic growth, starting with the arrival of the Georgia, Carolina, and Northern Railroad (later called the Seaboard Air Line) in 1892 and the founding of the Abbeville Cotton Mill Company in 1895. Other significant developments in the region included: the establishment of an African American school, called Ferguson Academy (later renamed Harbison Agricultural College), in 1868; the founding of Calhoun Falls at the junction of the Seaboard Air Line and Savannah Valley Railroad in the early 1890s; and the creation of Greenwood and McCormick Counties from portions of Abbeville County in 1899 and 1916. The Abbeville Press and Banner began as two weekly newspapers, the Abbeville Banner and Abbeville Press . Charles Henry Allen and Henry S. Kerr had established the Abbeville Banner in 1844. Its competitor, the Abbeville Independent Press (later known as the Abbeville Press ) began in 1853. Coleman C. Puckett and George W. Fant brought considerable experience to their jobs as publishers of the Abbeville Independent Press , Puckett having previously edited the Anderson Southern Rights Advocate and Fant having apprenticed with the Anderson Gazette . During the Civil War, the Abbeville Banner suspended publication. Hugh Wilson, Jr. and William Augustus Lee, the proprietors of the Abbeville Press , turned over publishing duties to a former business partner and enlisted in the Confederate States Army. Sadly, few issues of either newspaper have survived from the 1860s. After the war, Wilson and Lee bought the Abbeville Banner and merged it with the Abbeville Press to create the Abbeville Press and Banner . The first issue appeared on October 1, 1869. Like other white Democrat newspapers of the era, the Press and Banner steered a conservative course, celebrating the return of the “Bourbons,” or antebellum-era aristocrats, to political power in 1877 and championing the interests of agrarian elites. Throughout its existence, it faced only one true competitor, the Abbeville Medium , which existed from 1871 through 1923. Circulation rates for the Press and Banner generally hovered between 800 and 1,200 subscriptions. Several of the men who ran these newspapers were leaders in their communities. George W. Fant, who was briefly involved with the Independent Press , served as postmaster and director of the National Bank of Anderson. Hugh Wilson, Jr., in addition to editing and publishing the Press and Banner , served as director of and stockholder in the Abbeville Cotton Mill and was also an officer in the South Carolina Press Association. Fellow proprietor William Augustus Lee practiced real estate law, represented the Abbeville District in the South Carolina Constitutional Convention of 1865, and served as a state representative in 1865-66. Starting in the 1920s, the Abbeville Press and Banner underwent a series of changes, from adopting a triweekly publication schedule to shortening it

-------

Now that the data has been cleaned, we can begin pre-processing it. Let's [tokenize](https://www.nltk.org/_modules/nltk/tokenize/punkt.html) the essays into their individual components. First, we convert the `essay` column into a string. We'll next break the essays down into individual unigrams and then break those down into tokenized words. Read more about `unigrams` and `word tokenization` [here](https://www.nltk.org/api/nltk.tokenize.html)

In [6]:
df["essay"] = df["essay"].astype(str)

In [7]:
start = time.time()
df["unigrams"] = df["essay"].apply(nltk.word_tokenize)
print(time.time() - start)

4.268951177597046


Let's print out our unigrams to get a sense of what our code did.

In [8]:
text = df['unigrams'].apply(Text)
text.head()

0                                           (For, nearly, a, century, ,, the, Abbeville, Press, and, Banner, (, 1844-1925, ), reported, on, international, ,, national, ,, and, local, news, for, the, citizens, of, Abbeville, County, ,, located, in, the, upstate, region, of, South, Carolina, ., The, N.W, ., Ayer, &, Son, ’, s, American, Newspaper, Annual, described, Abbeville, County, simply, as, a, “, cotton, growing, district, ,, ”, which, while, true, on, the, surface, is, also, slightly, misleading, ., Although, the, cultivation, of, cotton, had, indeed, dominated, the, county, ’, s, economy, for, most, of, the, 19, th, century, ,, in, the, 1890s, Abbeville, was, undergoing, a, ...)
1            (The, short-lived, weekly, Abbeville, Bulletin, (, July-August, 1865, ), offers, a, glimpse, into, a, community, adapting, to, new, realities, under, challenging, circumstances, ., In, the, early, 1860s, ,, the, white, citizens, of, Abbeville, County, (, then, Abbeville, District, ), ,, South

Next we'll tokenize the sentences and apply part of speech tagging.

In [27]:
token_sent = df.essay.apply(nltk.word_tokenize)  
tagged_sent= token_sent.apply(nltk.pos_tag)
tagged_sent.head()

0                                                      [(For, IN), (nearly, RB), (a, DT), (century, NN), (,, ,), (the, DT), (Abbeville, NNP), (Press, NNP), (and, CC), (Banner, NNP), ((, (), (1844-1925, JJ), (), )), (reported, VBD), (on, IN), (international, JJ), (,, ,), (national, JJ), (,, ,), (and, CC), (local, JJ), (news, NN), (for, IN), (the, DT), (citizens, NNS), (of, IN), (Abbeville, NNP), (County, NNP), (,, ,), (located, VBN), (in, IN), (the, DT), (upstate, JJ), (region, NN), (of, IN), (South, NNP), (Carolina, NNP), (., .), (The, DT), (N.W, NNP), (., .), (Ayer, NNP), (&, CC), (Son, NNP), (’, NNP), (s, VBD), (American, JJ), (Newspaper, NNP), (Annual, NNP), (described, VBD), (Abbeville, NNP), (County, NNP), (simply, RB), (as, IN), (a, DT), (“, JJ), (cotton, NN), (growing, VBG), (district, NN), (,, ,), (”, NNP), (which, WDT), (while, IN), (true, JJ), (on, IN), (the, DT), (surface, NN), (is, VBZ), (also, RB), (slightly, RB), (misleading, VBG), (., .), (Although, IN), (the, DT), (cult

Now that we have pre-processed our `essays`, we can do basic things like get the count of words. e.g. 'century'

In [10]:
t = token_sent.apply(lambda x: nltk.Text(x).count('century'))
t

0       2
1       0
2       1
3       2
4       2
       ..
3675    0
3676    0
3677    0
3678    0
3679    0
Name: essay, Length: 3680, dtype: int64

More interesting, perhaps, is the fact that we can create a concordance of words and perform a search. In this example, we concordance search, grab 35 characters before / after word a particular word. Rows with `no matches` denote that the essay does not contain the text in question.

In [26]:
t = token_sent.apply(lambda x: nltk.Text(x).concordance("printer"))
t.head()

no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
 was elected to the Office of State Printer and moved to the state capitol , Sa
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
espectively . Hughes , trained as a printer , became co-owner of the Albuquerqu
no matches
Displaying 4 of 4 matches:
er achieving success as a newspaper printer in Baltimore , Matthew Brown ( 1774
 1774-1831 ) , Philadelphia-trained printer , hoped to launch a Federalist

no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
Lee Dillman ; John F. Moore was the printer . The weekly eight-page , six-colum
no matches
no matches
no matches
Displaying 1 of 1 matches:
rrell , veteran newspaper owner and printer , the paper maintained its independ
Displaying 1 of 1 matches:
rrell , veteran newspaper owner and printer , the paper maintained its independ
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
n 1882 . His brother Julius was the printer for both papers . In 1915 , Alfred 
af succeeded his father , Julius as printer . A cousin , Leander Graf , succeed
no matches
no matches
no matches
Displaying 1 of 1 matches:
ence with newspapers , as the first printer in the Wyoming Territory and publis
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches


no matches
no matches
Displaying 1 of 1 matches:
ant of Andrew Marschalk , the first printer in Mississippi Territory . George S
Displaying 1 of 1 matches:
ly Washington , D.C. , bookseller , printer , and mayor of the same name , and 
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
ge paper , after having worked as a printer on the Anaconda Standard and as a r
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
nt Phoenix in Brattleboro , in 1861 printer Charles A . Pierce moved across Ver
ng support for local improvements . Printer Otto R. Bennett published the Manch
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
 , 1895 . George M. Cooke , a local printer , had purchased the paper ’ s prede
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches


Displaying 1 of 1 matches:
 influenza , including the Echo ‘ s printer , Richard Masson , and the owner ’ 
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
bbs began his newspaper career as a printer ’ s apprentice to ambitious Robert 
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no ma

0    None
1    None
2    None
3    None
4    None
Name: essay, dtype: object

-------

Another aspect that thinks more deeply about extracting information from title essay content utilizes `spacy's` language models to isolate named identities such as people, organization, and locations within the unstructured content of the essays.

The code below extracts people and organizations from the title essays. The commented text also includes starter code to extract noun phrases or locations.

NOTE: This code takes a few minutes to run.

In [12]:
df['essay']= df['essay'].astype(str)
ppl_df = pd.DataFrame()


for row in range(len(df)):
    people = {}
    lccn = df.loc[row, "raw_lccn"]
    doc = nlp(df.loc[row, "essay"])
#     for np in doc.noun_chunks:
#         print(np.text)
    person = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
    lccn = str(lccn).strip()
    people[lccn] = [person, orgs]
    t = pd.DataFrame.from_dict(people, orient='index', columns=["people","organization"])
    ppl_df = pd.concat([ppl_df, t], ignore_index = True)

#     test['person'] = docs.ents.apply(ent.text if ent.label == 'PERSON')
#     orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
#     LOC = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
#     person = pd.DataFrame(person)
#     print(person)
#     test['person'] = person.loc[row, "essay"]


In [25]:
ppl_df.tail()

,people,organization
3675,"[bratska zajednica, Croatia, Kingdom of Serbs, Josep Broz Tito’s, Zajedničar, Svi Za Jednog-Jedan Za Sve]","[the National Croatian Society, the Croatian Association, Croatian Fraternal Union of America, CFU, Hrvatska, the National Croatian Society, the Croatian League of Illinois, the St. Joseph Society, the New Croatian Society, CFU, Home Office, Croats, the Socialist Federal Republic of Yugoslavia, CFU, Croatia, CFU, CFU, CFU, CFU, CFU, CFU, CFU, CFU, Radnička]"
3676,"[Dziennik, Dziennik Chicagoski, Dziennik Związkowy, Gazeta Polska Katolicka, Zasady w Polityce, Zapowiesci, Jezy]","[Harmony, Zwiazek Narodowo Polski, ZNP, the Polish National Alliance, ZNP, ZNP, ZNP, Zgoda: Wydania, Congress, ZNP]"
3677,"[Dziennik, Dziennik Chicagoski, Dziennik Związkowy, Gazeta Polska Katolicka, Zasady w Polityce, Zapowiesci, Jezy]","[Harmony, Zwiazek Narodowo Polski, ZNP, the Polish National Alliance, ZNP, ZNP, ZNP, Zgoda: Wydania, Congress, ZNP]"
3678,"[Dziennik, Dziennik Chicagoski, Dziennik Związkowy, Gazeta Polska Katolicka, Zasady w Polityce, Zapowiesci, Jezy]","[Harmony, Zwiazek Narodowo Polski, ZNP, the Polish National Alliance, ZNP, ZNP, ZNP, Zgoda: Wydania, Congress, ZNP]"
3679,"[Dziennik, Dziennik Chicagoski, Dziennik Związkowy, Gazeta Polska Katolicka, Zasady w Polityce, Zapowiesci, Jezy]","[Harmony, Zwiazek Narodowo Polski, ZNP, the Polish National Alliance, ZNP, ZNP, ZNP, Zgoda: Wydania, Congress, ZNP]"


In [14]:
df_cd = pd.concat([df, ppl_df], axis=1)

In [24]:
df_cd.head()

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url,unigrams,people,organization
0,"['Abbeville, S.C. : Charles H. Allen, 1847-1869.']",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']","For nearly a century, the Abbeville Press and Banner (1844-1925) reported on international, national, and local news for the citizens of Abbeville County, located in the upstate region of South Carolina. The N.W. Ayer & Son’s American Newspaper Annual described Abbeville County simply as a “cotton growing district,” which while true on the surface is also slightly misleading. Although the cultivation of cotton had indeed dominated the county’s economy for most of the 19 th century, in the 1890s Abbeville was undergoing a period of economic growth, starting with the arrival of the Georgia, Carolina, and Northern Railroad (later called the Seaboard Air Line) in 1892 and the founding of the Abbeville Cotton Mill Company in 1895. Other significant developments in the region included: the establishment of an African American school, called Ferguson Academy (later renamed Harbison Agricultural College), in 1868; the founding of Calhoun Falls at the junction of the Seaboard Air Line and Savannah Valley Railroad in the early 1890s; and the creation of Greenwood and McCormick Counties from portions of Abbeville County in 1899 and 1916. The Abbeville Press and Banner began as two weekly newspapers, the Abbeville Banner and Abbeville Press . Charles Henry Allen and Henry S. Kerr had established the Abbeville Banner in 1844. Its competitor, the Abbeville Independent Press (later known as the Abbeville Press ) began in 1853. Coleman C. Puckett and George W. Fant brought considerable experience to their jobs as publishers of the Abbeville Independent Press , Puckett having previously edited the Anderson Southern Rights Advocate and Fant having apprenticed with the Anderson Gazette . During the Civil War, the Abbeville Banner suspended publication. Hugh Wilson, Jr. and William Augustus Lee, the proprietors of the Abbeville Press , turned over publishing duties to a former business partner and enlisted in the Confederate States Army. Sadly, few issues of either newspaper have survived from the 1860s. After the war, Wilson and Lee bought the Abbeville Banner and merged it with the Abbeville Press to create the Abbeville Press and Banner . The first issue appeared on October 1, 1869. Like other white Democrat newspapers of the era, the Press and Banner steered a conservative course, celebrating the return of the “Bourbons,” or antebellum-era aristocrats, to political power in 1877 and championing the interests of agrarian elites. Throughout its existence, it faced only one true competitor, the Abbeville Medium , which existed from 1871 through 1923. Circulation rates for the Press and Banner generally hovered between 800 and 1,200 subscriptions. Several of the men who ran these newspapers were leaders in their communities. George W. Fant, who was briefly involved with the Independent Press , served as postmaster and director of the National Bank of Anderson. Hugh Wilson, Jr., in addition to editing and publishing the Press and Banner , served as director of and stockholder in the Abbeville Cotton Mill and was also an officer in the South Carolina Press Association. Fellow proprietor William Augustus Lee practiced real estate law, represented the Abbeville District in the South Carolina Constitutional Convention of 1865, and served as a state representative in 1865-66. Starting in the 1920s, the Abbeville Press and Banner underwent a series of changes, from adopting a triweekly publicat

In [17]:
final = pd.concat([df, ppl_df], axis=1, sort=False)

In [23]:
final.head()

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url,unigrams,people,organization
0,"['Abbeville, S.C. : Charles H. Allen, 1847-1869.']",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']","For nearly a century, the Abbeville Press and Banner (1844-1925) reported on international, national, and local news for the citizens of Abbeville County, located in the upstate region of South Carolina. The N.W. Ayer & Son’s American Newspaper Annual described Abbeville County simply as a “cotton growing district,” which while true on the surface is also slightly misleading. Although the cultivation of cotton had indeed dominated the county’s economy for most of the 19 th century, in the 1890s Abbeville was undergoing a period of economic growth, starting with the arrival of the Georgia, Carolina, and Northern Railroad (later called the Seaboard Air Line) in 1892 and the founding of the Abbeville Cotton Mill Company in 1895. Other significant developments in the region included: the establishment of an African American school, called Ferguson Academy (later renamed Harbison Agricultural College), in 1868; the founding of Calhoun Falls at the junction of the Seaboard Air Line and Savannah Valley Railroad in the early 1890s; and the creation of Greenwood and McCormick Counties from portions of Abbeville County in 1899 and 1916. The Abbeville Press and Banner began as two weekly newspapers, the Abbeville Banner and Abbeville Press . Charles Henry Allen and Henry S. Kerr had established the Abbeville Banner in 1844. Its competitor, the Abbeville Independent Press (later known as the Abbeville Press ) began in 1853. Coleman C. Puckett and George W. Fant brought considerable experience to their jobs as publishers of the Abbeville Independent Press , Puckett having previously edited the Anderson Southern Rights Advocate and Fant having apprenticed with the Anderson Gazette . During the Civil War, the Abbeville Banner suspended publication. Hugh Wilson, Jr. and William Augustus Lee, the proprietors of the Abbeville Press , turned over publishing duties to a former business partner and enlisted in the Confederate States Army. Sadly, few issues of either newspaper have survived from the 1860s. After the war, Wilson and Lee bought the Abbeville Banner and merged it with the Abbeville Press to create the Abbeville Press and Banner . The first issue appeared on October 1, 1869. Like other white Democrat newspapers of the era, the Press and Banner steered a conservative course, celebrating the return of the “Bourbons,” or antebellum-era aristocrats, to political power in 1877 and championing the interests of agrarian elites. Throughout its existence, it faced only one true competitor, the Abbeville Medium , which existed from 1871 through 1923. Circulation rates for the Press and Banner generally hovered between 800 and 1,200 subscriptions. Several of the men who ran these newspapers were leaders in their communities. George W. Fant, who was briefly involved with the Independent Press , served as postmaster and director of the National Bank of Anderson. Hugh Wilson, Jr., in addition to editing and publishing the Press and Banner , served as director of and stockholder in the Abbeville Cotton Mill and was also an officer in the South Carolina Press Association. Fellow proprietor William Augustus Lee practiced real estate law, represented the Abbeville District in the South Carolina Constitutional Convention of 1865, and served as a state representative in 1865-66. Starting in the 1920s, the Abbeville Press and Banner underwent a series of changes, from adopting a triweekly publicat

Finally, we'll save this data as a csv for external analysis:

In [21]:
final.to_csv('nlp_analysis.csv', index=False)